# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.

In [81]:
import pandas as pd
import numpy as np

In [132]:
data = pd.read_csv ('horse_data.csv',usecols = [0,1,2,3,4,5,10,22], 
                    names = ['Операция','Возраст','Ид','Температура','Пульс','Дыхание','Боль','Итог'])
data = data.replace('?', np.nan)
data.head()

,Операция,Возраст,Ид,Температура,Пульс,Дыхание,Боль,Итог
0,2,1,530101,38.50,66,28,5,2
1,1,1,534817,39.2,88,20,3,3
2,2,1,530334,38.30,40,24,3,1
3,1,9,5290409,39.10,164,84,2,2
4,2,1,530255,37.30,104,35,NaN,2


In [133]:
data['Операция'] = pd.to_numeric(data['Операция'], errors='coerce')
data['Температура'] = pd.to_numeric(data['Температура'], errors='coerce')
data['Пульс'] = pd.to_numeric(data['Пульс'], errors='coerce')
data['Дыхание'] = pd.to_numeric(data['Дыхание'], errors='coerce')
data.describe()

,Операция,Возраст,Ид,Температура,Пульс,Дыхание
count,299.000000,300.000000,3.000000e+02,240.000000,276.000000,242.000000
mean,1.397993,1.640000,1.085889e+06,38.167917,71.913043,30.417355
std,0.490305,2.173972,1.529801e+06,0.732289,28.630557,17.642231
min,1.000000,1.000000,5.184760e+05,35.400000,30.000000,8.000000
25%,1.000000,1.000000,5.289040e+05,37.800000,48.000000,18.500000
50%,1.000000,1.000000,5.303055e+05,38.200000,64.000000,24.500000
75%,2.000000,1.000000,5.347275e+05,38.500000,88.000000,36.000000
max,2.000000,9.000000,5.305629e+06,40.800000,184.000000,96.000000


Данные по возрасту есть по всем строкам.
По другим показателям данные не полные.
В столбце возраст есть какие-то ошибки, т.к. в базе говорится, что значения могут приниматься только 1 и 2, в датафрейме видим число 9. вероятно 9 эквивалентно 2.
Вероятно только у 25% лошадей температура в норме, судя по 25% квантилю.
Максимальное значение пулься 184 подозрительно, учитывая стандартное отклонение 28 при мреднем значении 72. Вероятно значение ошибочны.
Чаще всего по выбранным случаям лошадям поводили операцию

In [130]:
temp_range = data['Температура'].max() - data['Температура'].min()
pulse_range = data['Пульс'].max() - data['Пульс'].min()
print(temp_range)
print(pulse_range)

5.399999999999999
154.0


In [136]:
data['Возраст'].value_counts()

1    276
9     24
Name: Возраст, dtype: int64

In [138]:
q1 = data['Пульс'].quantile(0.25)
q3 = data['Пульс'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
data_wo_pulse = data[data['Пульс'].between(lower_bound, upper_bound, inclusive=True)].sort_values('Пульс')
data_wo_pulse

,Операция,Возраст,Ид,Температура,Пульс,Дыхание,Боль,Итог
130,1.0,1,528964,37.8,30.0,12.0,NaN,2
232,1.0,1,5262543,38.5,30.0,18.0,NaN,1
203,2.0,1,529685,37.2,36.0,9.0,2,1
276,1.0,1,534787,38.0,36.0,16.0,4,3
242,1.0,1,530354,NaN,36.0,32.0,5,2
...,...,...,...,...,...,...,...,...
148,1.0,1,529272,38.3,132.0,NaN,2,1
135,1.0,9,5287179,38.1,136.0,48.0,5,2
44,1.0,1,535407,35.4,140.0,24.0,4,3
103,1.0,9,5292489,38.0,140.0,68.0,3,1


In [139]:
pd.concat([data, data_wo_pulse]).drop_duplicates(keep=False)
# КАК СДЕЛАТЬ ТАК, ЧТОБЫ ОН УДАЛЯЛ ТОЛЬКО ВЫБРОСЫ, А ПУСТЫЕ ЗНАЧЕНИЯ НЕ ТРОГАЛ???

,Операция,Возраст,Ид,Температура,Пульс,Дыхание,Боль,Итог
3,1.0,9,5290409,39.1,164.0,84.0,2,2
5,2.0,1,528355,NaN,NaN,NaN,2,1
28,1.0,1,5279442,NaN,NaN,NaN,NaN,2
41,2.0,9,5288249,39.0,150.0,72.0,NaN,1
52,2.0,1,529483,NaN,NaN,NaN,1,1
55,1.0,9,5282839,38.6,160.0,20.0,3,2
56,1.0,1,528872,NaN,NaN,NaN,NaN,1
58,1.0,1,528298,NaN,NaN,20.0,5,2
74,1.0,9,5292929,NaN,NaN,NaN,NaN,2
93,2.0,1,530310,NaN,NaN,NaN,5,2


Много строчек удаляется по причине отсутствия значения по пульсу. 

In [140]:
q1 = data['Дыхание'].quantile(0.25)
q3 = data['Дыхание'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
data_wo_dyh = data[data['Дыхание'].between(lower_bound, upper_bound, inclusive=True)].sort_values('Дыхание')
data_wo_dyh

,Операция,Возраст,Ид,Температура,Пульс,Дыхание,Боль,Итог
190,1.0,1,528742,37.1,40.0,8.0,3,1
203,2.0,1,529685,37.2,36.0,9.0,2,1
140,1.0,1,529960,38.1,44.0,9.0,2,1
89,2.0,1,535381,38.3,42.0,10.0,1,1
162,2.0,1,535176,39.5,60.0,10.0,3,1
...,...,...,...,...,...,...,...,...
220,1.0,1,535029,39.2,88.0,58.0,5,3
20,1.0,1,530157,39.9,72.0,60.0,5,1
23,1.0,9,534998,38.3,130.0,60.0,2,1
185,1.0,1,534833,38.5,120.0,60.0,NaN,1


In [142]:
pd.concat([data, data_wo_dyh]).drop_duplicates(keep=False)['Дыхание'].value_counts()

68.0    3
80.0    3
70.0    2
90.0    2
96.0    2
84.0    2
88.0    1
66.0    1
72.0    1
Name: Дыхание, dtype: int64

Вылеты считаем допустимыми.

In [176]:
data = data_wo_pulse.dropna(thresh=6)

In [178]:
data['Температура'].fillna('37,8', inplace=True) # заменяем на нормальную температуру, если не мерили, значит поциент не жаловался :)
data['Боль'].fillna(data['Боль'].mode()[0], inplace=True) # смотрим какие лошадки к нам приходят с какими типами боли чаще всего
data['Пульс'].fillna(data.groupby('Возраст')['Пульс'].transform('mean'), inplace=True) # усредняем пульс в разрезе возраста, у молодых коней он может выше, так как удалили выбросы, если пульса нет, значит он среднестатистический
data['Дыхание'].fillna(data['Дыхание'].mean(), inplace=True) # дыхание не показательная величина, ее делаем средней
data = data.dropna(thresh=8) # осталась строка, где нет данных по ИД, коня лечили нелегально, удаляем строку. Еще нет в 1 строке данных по смерти, возможно сокрыли смерть, мы ее тоже не будем показывать.

In [179]:
data.describe()

,Операция,Возраст,Ид,Пульс,Дыхание
count,267.000000,267.000000,2.670000e+02,267.000000,267.000000
mean,1.411985,1.509363,9.586068e+05,70.247191,29.941858
std,0.493117,1.956987,1.363328e+06,26.169034,15.777045
min,1.000000,1.000000,5.213990e+05,30.000000,8.000000
25%,1.000000,1.000000,5.288970e+05,48.000000,20.000000
50%,1.000000,1.000000,5.302760e+05,64.000000,29.904762
75%,2.000000,1.000000,5.346220e+05,88.000000,35.500000
max,2.000000,9.000000,5.305629e+06,146.000000,96.000000


Удаляем неинформативные строки, допускаем, что до 10% можем удалить, т.е до 30 строк.

1: операция?
          1 = Да, была операция
          2 = Лечение прошло без хирургического вмешательства

  2: Возраст
          1 = взрослая лошадь
          2 = Молодой (<6 месяцев)

  3: Номер больницы
          - числовой идентификатор
          - номер дела, присвоенный лошади
            (не может быть уникальным, если лошадь лечится> 1 раз)

  4: ректальная температура
          - линейный
          - в градусах Цельсия.
          - Повышенная температура может возникнуть из-за инфекции.
          - возможно снижение температуры при позднем шоке.
          - нормальная температура 37,8
          - этот параметр обычно меняется по мере развития проблемы
               например. может начаться нормально, а затем повыситься из-за
                   поражение, возвращаясь через нормальный диапазон, как
                   лошадь в шоке
  5: пульс
          - линейный
          - частота пульса в ударах в минуту
          - отражает состояние сердца: 30-40 - нормально для взрослых
          - редко бывает ниже нормы, хотя у спортивных лошадей
            может иметь коэффициент 20-25
          - животные с болезненными поражениями или страдающие от шока кровообращения
            может иметь повышенную частоту сердечных сокращений

  6: частота дыхания
          - линейный
          - нормальная оценка от 8 до 10
          - полезность сомнительна из-за больших колебаний
          
11: боль - субъективное суждение об уровне боли лошади
           - возможные значения:
                1 = тревога, нет боли
                2 = депрессия
                3 = прерывистая легкая боль
                4 = периодическая сильная боль
                5 = постоянная сильная боль
           - НЕ следует рассматривать как упорядоченную или дискретную переменную!
           - В целом, чем больнее, тем больше вероятность того, что потребуется
             операция
           - предварительное лечение боли может до некоторой степени замаскировать уровень боли
           
           23: результат
           - что в итоге случилось с лошадью?
           - возможные значения:
                1 = жил
                2 = умер
                3 = был усыплен